# Accessing LMEC Collections via IIIF API

This notebook provides some tips for using Digital Commonwealth's IIIF API to query the LMEC collections portal and programmatically retrieve metadata about collections items.

### Understanding the IIIF API

Through the BPL/Digital Commonwealth, all of LMEC's maps are compliant with the International Image Interoperability Framework (IIIF). This means you can use IIIF APIs to retrieve Image and Presentation responses for any LMEC collection item.

#### Image API

A **Image API** request can return either *image metadata* or *a static image*.

Let's say we want to request metadata and an image for this recently-accessioned [map of summer resorts along the Boston & Maine Railroad](https://collections.leventhalmap.org/search/commonwealth:g158f6689).

The LMEC's API syntax for requesting image metadata is `BASE_URL` + `IMAGE_ID` + `/info.json`:

    # base URL
    https://iiif.digitalcommonwealth.org/iiif/2/

    # image information request
    https://iiif.digitalcommonwealth.org/iiif/2/IMAGE_ID/info.json

The image ID can be found by parsing the JSON data from any old collections item:


In [1]:
import json
import requests

item = requests.get("https://collections.leventhalmap.org/search/commonwealth:g158f6689.json")
print(item.json()['response']['document']['exemplary_image_ssi'])

commonwealth:7w62hz17g


We can append this image ID to the base URL to retrieve IIIF metadata:

In [17]:
base = "https://iiif.digitalcommonwealth.org/iiif/2/"
imageID = "commonwealth:7w62hz17g"

imageInfo = requests.get(base+imageID)

print(json.dumps(imageInfo.json(), indent=2))

{
  "@context": "http://iiif.io/api/image/2/context.json",
  "@id": "https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:7w62hz17g",
  "protocol": "http://iiif.io/api/image",
  "width": 7938,
  "height": 7111,
  "sizes": [
    {
      "width": 124,
      "height": 111
    },
    {
      "width": 248,
      "height": 222
    },
    {
      "width": 496,
      "height": 444
    },
    {
      "width": 992,
      "height": 889
    },
    {
      "width": 1985,
      "height": 1778
    },
    {
      "width": 3969,
      "height": 3556
    },
    {
      "width": 7938,
      "height": 7111
    }
  ],
  "tiles": [
    {
      "width": 1024,
      "height": 1024,
      "scaleFactors": [
        1,
        2,
        4,
        8,
        16,
        32,
        64
      ]
    }
  ],
  "profile": [
    "http://iiif.io/api/image/2/level2.json",
    {
      "formats": [
        "tif",
        "jpg",
        "gif",
        "png"
      ],
      "qualities": [
        "bitonal",
        "defa

The following syntax will return a static image:

    # full image as JPEG
    https://iiif.digitalcommonwealth.org/iiif/2/IMAGE_ID/full/full/0/default.jpg

And accessing it is as easy as combining a few sets of strings:

In [21]:
iiifSpec = "/full/full/0/default.jpg"

imageStatic = (base+imageID+iiifSpec)
print(imageStatic)

https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:7w62hz17g/full/full/0/default.jpg


By tweaking the `iiifSpec` variable, you can easily manipulate the image, adding parameters for size, rotation, quality, format, and more. See the [IIIF docs](https://iiif.io/api/image/3.0/) for more information.

#### Presentation API